# Результаты SRN, обученного на MJSynth + SynthText

In [1]:
import sys  
sys.path.insert(0, '/home/astrashnov/PaddleOCR/tools/')

import pandas as pd
from tqdm.notebook import tqdm
from eval_predictions import general_stat

%cd /home/astrashnov/PaddleOCR

/home/astrashnov/PaddleOCR


In [2]:
imgs_dirs = ('ic13/images', 'ic15/ch4_test_word_images_gt', 'iiit5k/test', 'svt/par', 'svt-perspective/par1', 'cute/images')
gt_paths = ('ic13/gt.txt', 'ic15/test_groundtruth_1811.txt', 'iiit5k/iiit5k_1k_test.txt',
            'svt/gt.txt', 'svt-perspective/gt.txt', 'cute/gt.txt')
names = ('ICDAR 2013', 'ICDAR 2015', 'IIIT5K', 'SVT', 'SVTP', 'CUTE')

In [3]:
results = {}
for name, img_dir, gt_path in zip(names, imgs_dirs, gt_paths):
    print(name)
    !python3 tools/infer_rec.py -c configs/rec/srn_MJSST_MC_Num.yml -o Global.pretrained_model=./output/rec/srn_MJSST_MC_Num/iter_epoch_8 Global.load_static_weights=false Global.infer_img=/home/pparamonov/data_clean/$img_dir > /dev/null
    stat = general_stat('output/rec/predicts_srn_MJSST_MC_Num.txt', '/home/pparamonov/data_clean/' + gt_path)
    results[name] = stat
    print()

ICDAR 2013
/home/astrashnov/anaconda3/envs/paddle_env/lib/python3.7/site-packages/skimage/morphology/_skeletonize.py:241: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  0, 1, 1, 0, 0, 1, 0, 0, 0], dtype=np.bool)
/home/astrashnov/anaconda3/envs/paddle_env/lib/python3.7/site-packages/skimage/morphology/_skeletonize.py:256: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.ht

In [4]:
df = pd.DataFrame.from_dict(results).transpose()
acc_columns = ['Accuracy (LC + UC, all words)',
               'Accuracy (LC + UC, long words)',
               'Accuracy (MC, all words)',
               'Accuracy (MC, long words)',]
all_columns = df.columns
for column in acc_columns:
    df[column] *= 100
df = df.transpose()
df.style.format('{:.4f}').format('{:.2f}%', subset=pd.IndexSlice[acc_columns, :])

,ICDAR 2013,ICDAR 2015,IIIT5K,SVT,SVTP,CUTE
"Accuracy (LC + UC, all words)",23.10%,76.42%,88.90%,0.00%,0.00%,14.24%
"Accuracy (LC + UC, long words)",12.90%,70.31%,72.44%,0.00%,0.00%,7.14%
"Accuracy (MC, all words)",94.75%,76.42%,88.90%,88.56%,78.60%,77.78%
"Accuracy (MC, long words)",94.62%,70.31%,72.44%,90.70%,90.24%,57.14%
"Normalized edit distance 1 (LC + UC, all words)",0.4926,0.1060,0.0402,1.0043,1.0027,0.7653
"Normalized edit distance 1 (LC + UC, long words)",0.4054,0.0886,0.0818,1.0021,1.0024,0.7310
"Normalized edit distance 1 (MC, all words)",0.0210,0.1060,0.0402,0.0476,0.0982,0.1053
"Normalized edit distance 1 (MC, long words)",0.0192,0.0886,0.0818,0.0366,0.0683,0.2283
"Normalized edit distance 2 (LC + UC, all words)",0.4795,0.1018,0.0432,1.0032,1.0021,0.8045
"Normalized edit distance 2 (LC + UC, long words)",0.4017,0.0904,0.1178,1.0022,1.0024,0.6923
